# Gradient boosting practice


This task will use the open source `boston` dataset from `sklearn.datasets`. The last 25% of objects will be left for quality control, dividing `X` and `y` into `X_train`, `y_train` and `X_test`, `y_test`.

The goal of the assignment will be to implement a simple version of gradient boosting over regression trees for the case of a quadratic loss function.

In [1]:
from sklearn import datasets
%pylab inline
#Populating the interactive namespace from numpy and matplotlib
boston = datasets.load_boston()
print(boston.DESCR)
print(boston.keys())
#print("feature names: {}".format(boston.feature_names))
#print("target names: {names}".format(names = boston.target))
from pandas import DataFrame
boston_frame = DataFrame(boston.data)
boston_frame.columns = boston.feature_names
boston_frame['target'] = boston.target
boston_frame.head()

C:\Python\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Populating the interactive namespace from numpy and matplotlib
.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX     

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [2]:
print('df shape:', boston_frame.shape)
X = boston_frame.drop('target', 1)
y = boston_frame['target']
X

df shape: (506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [3]:
from sklearn.model_selection import train_test_split
(X_train, X_test, 
 y_train, y_test) = train_test_split(X, y, 
                                     test_size=0.25, 
                                     shuffle=False)

print ("Size after split:\nX_train: {}, y_train: {},\nX_test: {}, y_test: {}.".format(X_train.shape, 
                                                                                      len(y_train), X_test.shape, len(y_test)))
y_test

Size after split:
X_train: (379, 13), y_train: 379,
X_test: (127, 13), y_test: 127.


379    10.2
380    10.4
381    10.9
382    11.3
383    12.3
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Name: target, Length: 127, dtype: float64

In [4]:
#using another approach for data split:
boston = datasets.load_boston()
X = boston.data 
y = boston.target

In [5]:
splitting=len(X)-round(len(X)*0.25)

X_train=X[0:splitting,:]
X_test=X[splitting:,:]
y_train=y[0:splitting]
y_test=y[splitting:]
print("X, {}, y,{}, Xtrain, {}, Xtest, {}, ytrain, {}, ytest, {}".format(len(X),len(y), 
                                                                         len(X_train), len(X_test), 
                                                                         len(y_train), len(y_test)))


X, 506, y,506, Xtrain, 380, Xtest, 126, ytrain, 380, ytest, 126


In [6]:
y_test

array([10.4, 10.9, 11.3, 12.3,  8.8,  7.2, 10.5,  7.4, 10.2, 11.5, 15.1,
       23.2,  9.7, 13.8, 12.7, 13.1, 12.5,  8.5,  5. ,  6.3,  5.6,  7.2,
       12.1,  8.3,  8.5,  5. , 11.9, 27.9, 17.2, 27.5, 15. , 17.2, 17.9,
       16.3,  7. ,  7.2,  7.5, 10.4,  8.8,  8.4, 16.7, 14.2, 20.8, 13.4,
       11.7,  8.3, 10.2, 10.9, 11. ,  9.5, 14.5, 14.1, 16.1, 14.3, 11.7,
       13.4,  9.6,  8.7,  8.4, 12.8, 10.5, 17.1, 18.4, 15.4, 10.8, 11.8,
       14.9, 12.6, 14.1, 13. , 13.4, 15.2, 16.1, 17.8, 14.9, 14.1, 12.7,
       13.5, 14.9, 20. , 16.4, 17.7, 19.5, 20.2, 21.4, 19.9, 19. , 19.1,
       19.1, 20.1, 19.9, 19.6, 23.2, 29.8, 13.8, 13.3, 16.7, 12. , 14.6,
       21.4, 23. , 23.7, 25. , 21.8, 20.6, 21.2, 19.1, 20.6, 15.2,  7. ,
        8.1, 13.6, 20.1, 21.8, 24.5, 23.1, 19.7, 18.3, 21.2, 17.5, 16.8,
       22.4, 20.6, 23.9, 22. , 11.9])

## Task 1

**Boosting** is a method of building compositions of basic algorithms by sequentially adding a new algorithm to the current composition with a certain coefficient.

Gradient boosting trains each new algorithm so that it approximates the error anti-gradient over composition responses on the training set. Similar to function minimization using gradient descent, in gradient boosting we tweak the composition by changing the algorithm in the direction of the error anti-gradient.


The function has the form L(a(x), y) = [a(x) - y]^2.

You need to differentiate with respect to a(x). 

## Task 2

Get an array for `DecisionTreeRegressor` objects (we will use them as basic algorithms) and for real numbers (these will be coefficients in front of the basic algorithms).

In the loop from, train 50 decision trees sequentially with the parameters `max_depth=5` and `random_state=42` (other parameters are default). Boosting often uses hundreds or thousands of trees, but we will limit ourselves to 50 so that the algorithm runs faster and is easier to debug (because the goal of the task is to figure out how the method works). Each tree must be trained on the same set of objects, but the answers that the tree learns to predict will change in accordance with the rule obtained in task 1.

Try to always start with a factor of 0.9. It is usually justified to choose a coefficient much smaller - about 0.05 or 0.1, but since in our training example, there will be only 50 trees on a standard dataset, let's take a larger step for a start.

In the process of implementing the training, you will need a function that will calculate the forecast of the composition of trees built at the moment on the sample `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

```

The same function will help you get a prediction on the control sample and evaluate the performance of your algorithm using `mean_squared_error` в `sklearn.metrics`. 

Raise the result to the power of 0.5 to get `RMSE`.

In [7]:
from sklearn import model_selection, metrics, tree 
import numpy as np

def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]


base_algorithms_list = []
coefficients_list = []
s = y_train

for i in range(50):
    #clf = tree.DecisionTreeClassifier(max_depth=5, random_state=42)
    alg = tree.DecisionTreeRegressor(max_depth=5, random_state=42) 
    alg.fit(X_train, s)
    coefficients_list.append(0.9) 
    base_algorithms_list.append(alg) 
    s=y_train-gbm_predict(X_train)
    
mse = metrics.mean_squared_error(y_test, gbm_predict(X_test)) 
print(mse ** 0.5)

5.455221764927312


In [8]:
def write_answer(answer, name):
    
    with open(name, "w") as fout:
        fout.write(str(answer))
        
write_answer((mse ** 0.5), "grad_boost_answer1.txt")

## Task 3

Try to reduce the weight before each algorithm with each next iteration according to the formula `0.9 / (1.0 + i)`, where `i` is the iteration number (from 0 to 49). Use the performance of the algorithm as **answer in point 3**.

In reality, the following step selection strategy is often used: as soon as an algorithm is chosen, we select the coefficient in front of it by a numerical optimization method in such a way that the deviation from the correct answers is minimal. We will not suggest that you implement this for the task, but we recommend that you try to figure out such a strategy and implement it on occasion for yourself.

In [9]:
base_algorithms_list = []
coefficients_list = []
s = y_train

for i in range(50):
    #clf = tree.DecisionTreeClassifier(max_depth=5, random_state=42)
    alg = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    alg.fit(X_train, s)
    coefficients_list.append(0.9 / (1.0 + i)) 
    base_algorithms_list.append(alg) 
    s=y_train-gbm_predict(X_train)
    
mse = metrics.mean_squared_error(y_test, gbm_predict(X_test)) 
print(mse ** 0.5)
write_answer((mse ** 0.5), "grad_boost_answer2.txt")

5.241033584774468


## Task 4

The method implemented - gradient boosting over trees - is very popular in machine learning. It is present both in the `sklearn` library itself and in the third-party `XGBoost` library, which has its own Python interface. In practice, `XGBoost` performs noticeably better than `GradientBoostingRegressor` from `sklearn`, but you can use any implementation for this assignment.

Investigate whether gradient boosting overfits as the number of iterations increases (and think about why) and also as the depth of the trees increases. 

class sklearn.ensemble.GradientBoostingRegressor
(*, loss='squared_error', learning_rate=0.1, n_estimators=100, 
subsample=1.0, criterion='friedman_mse', min_samples_split=2, 
min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, 
init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, 
warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

loss{‘squared_error’, ‘absolute_error’, ‘huber’, ‘quantile’}, default=’squared_error’
Loss function to be optimized. ‘squared_error’ refers to the squared error for regression. ‘absolute_error’ refers to the absolute error of regression and is a robust loss function. ‘huber’ is a combination of the two. ‘quantile’ allows quantile regression (use alpha to specify the quantile).

n_estimators int, default=100
The number of boosting stages to perform. Gradient boosting is fairly robust to over-fitting so a large number usually results in better performance.

criterion{‘friedman_mse’, ‘squared_error’, ‘mse’, ‘mae’}, default=’friedman_mse’
The function to measure the quality of a split. Supported criteria are “friedman_mse” for the mean squared error with improvement score by Friedman, “squared_error” for mean squared error, and “mae” for the mean absolute error. The default value of “friedman_mse” is generally the best as it can provide a better approximation in some cases.

initestimator or ‘zero’, default=None
An estimator object that is used to compute the initial predictions. init has to provide fit and predict. If ‘zero’, the initial raw predictions are set to zero. By default a DummyEstimator is used, predicting either the average target value (for loss=’squared_error’), or a quantile for the other losses.

In [10]:
from sklearn import ensemble
alg = ensemble.GradientBoostingRegressor(n_estimators=10)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("n_estimators=10")
print("test quality = ", mse ** 0.5)

alg = ensemble.GradientBoostingRegressor(n_estimators=100)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("n_estimators=100")
print("test quality = ", mse ** 0.5)

alg = ensemble.GradientBoostingRegressor(n_estimators=500)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("n_estimators=500")
print("test quality = ", mse ** 0.5)

alg = ensemble.GradientBoostingRegressor(n_estimators=1000)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("n_estimators=1000")
print("test quality = ", mse ** 0.5)

alg = ensemble.GradientBoostingRegressor(n_estimators=2000)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("n_estimators=2000")
print("test quality = ", mse ** 0.5)

alg = ensemble.GradientBoostingRegressor(n_estimators=3000)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("n_estimators=3000")
print("test quality = ", mse ** 0.5)


alg = ensemble.GradientBoostingRegressor(max_depth=2)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("max_depth=2")
print("test quality = ", mse ** 0.5)

alg = ensemble.GradientBoostingRegressor(max_depth=5)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("max_depth=5")
print("test quality = ", mse ** 0.5)

alg = ensemble.GradientBoostingRegressor(max_depth=10)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("max_depth=10")
print("test quality = ", mse ** 0.5)

alg = ensemble.GradientBoostingRegressor(max_depth=20)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("max_depth=20")
print("test quality = ", mse ** 0.5)

alg = ensemble.GradientBoostingRegressor(max_depth=50)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("max_depth=50")
print("test quality = ", mse ** 0.5)

alg = ensemble.GradientBoostingRegressor(max_depth=100)
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("max_depth=100")
print("test quality = ", mse ** 0.5)

answer3="2 3"
write_answer(answer3, "grad_boost_answer3.txt")



n_estimators=10
test quality =  7.285454097125206
n_estimators=100
test quality =  4.504648791660076
n_estimators=500
test quality =  4.425647100253689
n_estimators=1000
test quality =  4.465183540025864
n_estimators=2000
test quality =  4.532192436541141
n_estimators=3000
test quality =  4.447336161608743
max_depth=2
test quality =  4.555488352072528
max_depth=5
test quality =  4.721408610684704
max_depth=10
test quality =  5.868081532878581
max_depth=20
test quality =  5.727168923863321
max_depth=50
test quality =  5.901620839416651
max_depth=100
test quality =  5.586913572014026


## Task 5

Compare the performance you get with gradient boosting to that of linear regression.

To do this, train the `LinearRegression` from `sklearn.linear_model` (with default parameters) on the training set and evaluate for the predictions of the resulting algorithm on the `RMSE` test set. The quality received is the answer in **point 5**.

In this example, the performance of a simple model should have been worse, but do not forget that this is not always the case. In the assignments for this course, you will also find an example of the reverse situation.

In [11]:
from sklearn import linear_model
alg = linear_model.LinearRegression()
alg.fit(X_train, y_train)
mse = metrics.mean_squared_error(y_test, alg.predict(X_test)) 
print("test quality = ", mse ** 0.5)
write_answer(mse ** 0.5, "grad_boost_answer4.txt")

test quality =  7.819688142087445
